In [1]:
import time
start_time = time.time()

In [2]:
import numpy as np
import pandas as pd 
import spotipy
import spotipy.oauth2
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import statistics
from scipy.stats import kurtosis
from scipy.stats import skew

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.externals import joblib

def f_mean(row):
    return(np.mean(row))
def f_median(row):
    return(np.median(row))
def f_std(row):
     return(np.std(row))
def f_min(row):
     return(np.min(row))
def f_max(row):
     return(np.max(row))
def f_80Percentile(row):
    return(np.percentile(row, 80))
def f_kurtosis(row):
    return(kurtosis(row))
def f_skew(row):
    return(skew(row))

In [3]:
CLIENT_ID = '23888f5deee6452db5d78bd2d1091da5'
CLIENT_SECRET = 'f6ecea7a89d24b02b2c969745b88a9db'

client_credentials_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
#dat = pd.read_csv('cleanedDataSet_final.csv')

In [5]:
#print(dat.columns.values)
#dat.head(5)

In [6]:
def generate_df(sp_feature_dict):
	df = pd.DataFrame.from_dict(sp_feature_dict) #convert dict to pandas table

	df = df.loc[:,['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence']] #clean up table (extract columns we need)
	return(df)

In [7]:
#Test with a random song.
def search_spotify(Artist, Track):
	
	'----- Grab authentication -----'
	CLIENT_ID = '23888f5deee6452db5d78bd2d1091da5'
	CLIENT_SECRET = 'f6ecea7a89d24b02b2c969745b88a9db'

	client_credentials_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
	sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

	q = 'artist:{} track:{}'.format(Artist, Track) #get artist_name and track_name from html 
	results = sp.search(q, type='track')
	#print('aaaaaa') used for debugging purposes
	#print(results)
	return(results)

In [8]:
results = search_spotify('Grouplove', 'Spun')

In [9]:
	track_uri = results['tracks']['items'][0]['uri'] #nested dict: get tracks, items of tracks, 1st item, and then uri of the 1st item
	artist_uri = results['tracks']['items'][0]['artists'][0]['uri'] #gets artist info, needed to grab the genre

### We have the uris now

In [10]:
	#track_popularity = results['tracks']['items'][0]['popularity'] #get popularity of track
	track_genre = sp.artist(artist_uri)['genres'] #get genre of artist
	track_features = sp.audio_features(track_uri) #grab dict of audio features
	track_name = results['tracks']['items'][0]['name'] #get name of track
	new_track_df = generate_df(track_features) #now have pandas df with relevant columns, this will be used for output
	if not track_genre:
		track_genre = 'Unknown'
	else:
		track_genre = track_genre[0] #to prevent index error for genre selection
	#new_track_df['popularity'] = track_popularity
	new_track_df['genre'] = track_genre
	new_track_df['song'] = track_name
	new_track_df['uri'] = track_uri

In [11]:
new_track_df

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence,genre,song,uri
0,0.00262,0.424,205613,0.851,0.000022,7,0.232,-6.143,1,0.0385,155.018,0.397,indie folk,Spun,spotify:track:6WgvvNUermK2Xad0jRPxXr


### Now we need to append the timbre aggregation values as new columns

In [12]:
%%time
timbre_features = ['timbre_all_1', 'timbre_all_2',
       'timbre_all_3', 'timbre_all_4', 'timbre_all_5', 'timbre_all_6',
       'timbre_all_7', 'timbre_all_8', 'timbre_all_9', 'timbre_all_10',
       'timbre_all_11', 'timbre_all_12']

timbre_all_df = pd.DataFrame(index = range(len(new_track_df)), columns = timbre_features)

curr_uri = new_track_df['uri'][0]
curr_song = sp.audio_analysis(curr_uri)
curr_segments = curr_song['segments'] #get into segments (time intervals). Each song diff length

init_timbres_all = [[] for _ in range(12)] #initialize 12 lists for median (find median each list)

for j in range(0, len(curr_segments)):
        curr_timbres_list = curr_segments[j]['timbre'] #grab timbres as list
        
        init_timbres_all[0].append(curr_timbres_list[0])
        init_timbres_all[1].append(curr_timbres_list[1])
        init_timbres_all[2].append(curr_timbres_list[2])
        init_timbres_all[3].append(curr_timbres_list[3])
        init_timbres_all[4].append(curr_timbres_list[4])
        init_timbres_all[5].append(curr_timbres_list[5])
        init_timbres_all[6].append(curr_timbres_list[6])
        init_timbres_all[7].append(curr_timbres_list[7])
        init_timbres_all[8].append(curr_timbres_list[8])
        init_timbres_all[9].append(curr_timbres_list[9])
        init_timbres_all[10].append(curr_timbres_list[10])
        init_timbres_all[11].append(curr_timbres_list[11])
        
timbre_all_df['timbre_all_1'][0] = init_timbres_all[0]
timbre_all_df['timbre_all_2'][0] = init_timbres_all[1]
timbre_all_df['timbre_all_3'][0] = init_timbres_all[2]
timbre_all_df['timbre_all_4'][0] = init_timbres_all[3]
timbre_all_df['timbre_all_5'][0] = init_timbres_all[4]
timbre_all_df['timbre_all_6'][0] = init_timbres_all[5]
timbre_all_df['timbre_all_7'][0] = init_timbres_all[6]
timbre_all_df['timbre_all_8'][0] = init_timbres_all[7]
timbre_all_df['timbre_all_9'][0] = init_timbres_all[8]
timbre_all_df['timbre_all_10'][0] = init_timbres_all[9]
timbre_all_df['timbre_all_11'][0] = init_timbres_all[10]
timbre_all_df['timbre_all_12'][0] = init_timbres_all[11]

new_track_df = pd.concat([new_track_df, timbre_all_df], axis=1)
j = 0
for i in range(15, len(new_track_df.columns)):
    current_column  = new_track_df.iloc[:,i]
    new_track_df['mean_timbre{}'.format(j+1)] = current_column.apply(f_mean)
    new_track_df['median_timbre{}'.format(j+1)] = current_column.apply(f_median)
    new_track_df['std_timbre{}'.format(j+1)] = current_column.apply(f_std)
    new_track_df['min_timbre{}'.format(j+1)] = current_column.apply(f_min)
    new_track_df['max_timbre{}'.format(j+1)] = current_column.apply(f_max)
    new_track_df['range_timbre{}'.format(j+1)] = new_track_df['max_timbre{}'.format(j+1)]-new_track_df['min_timbre{}'.format(j+1)]
    new_track_df['80Percentile_timbre{}'.format(j+1)] = current_column.apply(f_80Percentile)
    new_track_df['kurtosis_timbre{}'.format(j+1)] = current_column.apply(f_kurtosis)
    new_track_df['skewness_timbre{}'.format(j+1)] = current_column.apply(f_skew)
    j += 1

CPU times: user 195 ms, sys: 13 ms, total: 208 ms
Wall time: 1.6 s


In [13]:
new_track_df.head(2)

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,...,skewness_timbre11,mean_timbre12,median_timbre12,std_timbre12,min_timbre12,max_timbre12,range_timbre12,80Percentile_timbre12,kurtosis_timbre12,skewness_timbre12
0,0.00262,0.424,205613,0.851,0.000022,7,0.232,-6.143,1,0.0385,...,0.045619,-4.902242,-4.628,16.111457,-54.198,48.012,102.21,7.587,0.107498,-0.123576


### We have the clean test row now. Use: random_forest_model from pkl files

In [14]:
local_output = ['ee', 'br',
       'my', 'cz', 'pe', 'gb', 'ie', 'se', 'sk', 'co', 'sg', 'ec', 'es', 'no',
       'dk', 'be', 'lu', 'do', 'nz', 'de', 'hu', 'cr', 'jp', 'hk', 'lt', 'pl',
       'it', 'hn', 'mx', 'ph', 'cl', 'is', 'ca', 'ar', 'at', 'ch', 'tr', 'py',
       'tw', 'gt', 'sv', 'us', 'fr', 'global', 'nl', 'bo', 'lv', 'gr', 'pa',
       'uy', 'au', 'pt', 'fi']

In [15]:
%%time
model_list = []
for i in local_output:
    model_list.append(joblib.load('{}_model.pkl'.format(i)))

CPU times: user 5.22 s, sys: 383 ms, total: 5.6 s
Wall time: 5.75 s


In [16]:
rel_features = ['median_timbre4', 'range_timbre7', '80Percentile_timbre9',
       'skewness_timbre3', '80Percentile_timbre3',
       '80Percentile_timbre11', 'range_timbre5', 'min_timbre2',
       'skewness_timbre4', 'mean_timbre8', 'mean_timbre7',
       'median_timbre12', 'min_timbre5', 'kurtosis_timbre9',
       'std_timbre5', 'skewness_timbre7', 'range_timbre6',
       'median_timbre9', 'loudness', 'kurtosis_timbre1', 'liveness',
       'min_timbre9', '80Percentile_timbre10', 'min_timbre8',
       'kurtosis_timbre11', '80Percentile_timbre8', 'valence',
       'max_timbre8', 'speechiness', 'kurtosis_timbre10',
       'range_timbre11', 'skewness_timbre1', 'min_timbre11',
       'mean_timbre1', 'mean_timbre2', 'kurtosis_timbre12',
       '80Percentile_timbre1', 'range_timbre1', 'kurtosis_timbre5',
       'std_timbre1', 'instrumentalness', 'min_timbre7', 'std_timbre7',
       'danceability', '80Percentile_timbre4', 'energy',
       'skewness_timbre11', 'std_timbre8', 'min_timbre6',
       'kurtosis_timbre4', 'mean_timbre9', 'skewness_timbre2',
       '80Percentile_timbre2', 'duration_ms', 'kurtosis_timbre8',
       'std_timbre6', 'range_timbre3', 'std_timbre9', 'max_timbre2',
       'std_timbre10']

In [17]:
#df_features = new_track_df[rel_features]

In [18]:
new_track_df = new_track_df[new_track_df.columns.difference(['timbre_all_1', 'timbre_all_2', 'timbre_all_3', 
                                                             'timbre_all_4', 'timbre_all_5', 'timbre_all_6',
                                                             'timbre_all_7', 'timbre_all_8', 'timbre_all_9', 
                                                             'timbre_all_10', 'timbre_all_11', 'timbre_all_12', 
                                                             'song', 'uri', 'genre'])]

In [19]:
#X_norm = normalize(df_features)
X_norm = normalize(new_track_df)

local_output = ['ee', 'br',
       'my', 'cz', 'pe', 'gb', 'ie', 'se', 'sk', 'co', 'sg', 'ec', 'es', 'no',
       'dk', 'be', 'lu', 'do', 'nz', 'de', 'hu', 'cr', 'jp', 'hk', 'lt', 'pl',
       'it', 'hn', 'mx', 'ph', 'cl', 'is', 'ca', 'ar', 'at', 'ch', 'tr', 'py',
       'tw', 'gt', 'sv', 'us', 'fr', 'global', 'nl', 'bo', 'lv', 'gr', 'pa',
       'uy', 'au', 'pt', 'fi']

In [20]:
region_dict = {}
for i in range(0, len(local_output)):
    curr_country = local_output[i]
    curr_model = model_list[i]
    region_dict[curr_country] = curr_model.predict(X_norm)[0]

In [21]:
print(time.time() - start_time)
region_dict

17.255165815353394


{'ar': True,
 'at': True,
 'au': False,
 'be': True,
 'bo': False,
 'br': True,
 'ca': True,
 'ch': True,
 'cl': True,
 'co': True,
 'cr': False,
 'cz': True,
 'de': False,
 'dk': False,
 'do': True,
 'ec': True,
 'ee': False,
 'es': True,
 'fi': False,
 'fr': True,
 'gb': False,
 'global': True,
 'gr': True,
 'gt': True,
 'hk': False,
 'hn': True,
 'hu': False,
 'ie': True,
 'is': False,
 'it': True,
 'jp': True,
 'lt': True,
 'lu': True,
 'lv': True,
 'mx': True,
 'my': True,
 'nl': True,
 'no': False,
 'nz': True,
 'pa': True,
 'pe': True,
 'ph': True,
 'pl': False,
 'pt': False,
 'py': True,
 'se': True,
 'sg': True,
 'sk': False,
 'sv': False,
 'tr': True,
 'tw': True,
 'us': True,
 'uy': True}